In [1]:
import pennylane as qml
from pennylane import numpy as np

In [2]:
def qft(wires):
    n = len(wires)
    for i in range(n):
        qml.Hadamard(wires=wires[i])
        for j in range(i + 1, n):
            qml.ControlledPhaseShift(np.pi / 2 ** (j - i), wires=[wires[j], wires[i]])
    for i in range(n // 2):
        qml.SWAP(wires=[wires[i], wires[n - i - 1]])

In [3]:
def iqft(wires):
    n = len(wires)
    for i in range(n // 2):
        qml.SWAP(wires=[wires[i], wires[n - i - 1]])
    for i in reversed(range(n)):
        for j in reversed(range(i + 1, n)):
            qml.ControlledPhaseShift(-np.pi / 2 ** (j - i), wires=[wires[j], wires[i]])
        qml.Hadamard(wires=wires[i])

In [4]:
def qpe_for_s(n, wires):
    control_wires = wires[:n]
    target_wires = wires[n:]
    
    for i in range(n):
        qml.Hadamard(wires=control_wires[i])

    qml.ControlledPhaseShift(np.pi / 2, wires=[control_wires[0], target_wires[0]])
    qml.ControlledPhaseShift(np.pi, wires=[control_wires[1], target_wires[1]])

    iqft(control_wires)

In [5]:
# Number of qubits
n = 4

# Define device
dev = qml.device("default.qubit", wires=2 * n, shots=1024)

# Define QPE function
@qml.qnode(dev)
def qpe_circuit():
    qpe_for_s(n, wires=list(range(2 * n)))
    return qml.sample(wires=list(range(n)))

# Run simulation
samples = qpe_circuit()
unique, counts = np.unique(samples, return_counts=True)

# Print results
print("Measurement results:")
for state, count in zip(unique, counts):
    print(f"{bin(state)[2:].zfill(n)} (binary) -> {count} counts")

Measurement results:
0000 (binary) -> 4096 counts
